In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Define the core features (frozen schema)
core_features = [
    "gpt2_perplexity", "type_token_ratio", "repeated_3gram_ratio",
    "unique_2grams", "unique_3grams", "sentence_length_std",
    "sentence_length_entropy", "burstiness_token_std", "burstiness_token_var",
    "pos_transition_entropy", "punctuation_ratio", "avg_word_length",
    "flesch_reading_ease", "pos_ratio_X", "repeated_2gram_ratio"
]



In [3]:
# TODO: Update this path to where your CSV actually lives.
file_path = r"C:\Users\marco\Desktop\Thesis\data\processed\train_sample_baseline.csv"

# Load the DataFrame
df = pd.read_csv(file_path)

# Inspect the first few rows to verify it loaded correctly
df.head()


,id,adv_source_id,source_id,model,decoding,repetition_penalty,attack,domain,title,prompt,...,pos_transition_entropy,gpt2_perplexity,avg_sentence_length_v2,sentence_length_entropy,repeated_2gram_ratio,unique_2grams,repeated_3gram_ratio,unique_3grams,burstiness_token_var,burstiness_token_std
0,44680e7b-a3c5-4984-9dae-dc39371c4d3d,2f47f011-2a2a-47fb-8380-81bbfd3aea9f,2f47f011-2a2a-47fb-8380-81bbfd3aea9f,human,NaN,NaN,article_deletion,reddit,Anyone else have or had this?,NaN,...,5.734448,35.831509,14.7500,3.444685,0.048780,154.0,0.006135,162.0,3.508151,1.873006
1,1022b415-d62d-4542-8b22-c32d6689b422,76cf3c61-aa65-49a3-b3a5-6fdba510104d,76cf3c61-aa65-49a3-b3a5-6fdba510104d,human,NaN,NaN,alternative_spelling,books,Father Malachy's Miracle,NaN,...,5.639839,30.296240,23.0625,3.921559,0.057377,314.0,0.021918,344.0,14.939022,3.865103
2,2dcca159-a72f-425b-a42d-8af9aa77b1ed,17548d1a-9ad1-4d65-b97e-6d022f1413bd,a821b653-e817-4478-bddc-8012757258e7,mistral,greedy,no,zero_width_space,reddit,Do i have to wait another 20 years?,The following is the full text of a post title...,...,3.807238,6.557842,178.0000,0.634578,0.000000,0.0,0.000000,0.0,0.000000,0.000000
3,444307a2-81a6-43c3-98eb-75bd4db300a6,cdeaa1ba-cdcd-48b6-a711-2cd5e8f630ae,cdeaa1ba-cdcd-48b6-a711-2cd5e8f630ae,human,NaN,NaN,number,news,Ireland surge past Scots,NaN,...,5.628694,55.334324,25.0800,4.470719,0.066895,535.0,0.015464,572.0,9.503216,3.082729
4,a1bd9e19-e76b-47de-b147-8319409506a2,72c85ea8-e1ae-4a13-a75f-99022c62d5b2,72c85ea8-e1ae-4a13-a75f-99022c62d5b2,human,NaN,NaN,number,wiki,Saraswati Rane,NaN,...,5.481764,40.634918,20.8750,2.872306,0.116129,134.0,0.064935,144.0,1.825852,1.351241


In [4]:
df["binary_label"] = df["model"].apply(lambda m: "human" if m == "human" else "artificial")
df["binary_label_code"] = (df["binary_label"] == "artificial").astype(int)

In [5]:
# Extract X and y
X = df[core_features]
y = df["binary_label_code"]  # or whatever your target column is named

# Quick sanity check
print("Number of features:", len(core_features))
print("Shape of X:", X.shape)
print("Series y unique values:", y.unique())

Number of features: 15
Shape of X: (16000, 15)
Series y unique values: [0 1]


In [6]:
# Initialize a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# 5‐fold CV on the full feature set
baseline_scores = cross_val_score(clf, X, y, cv=5, scoring="roc_auc")
baseline_mean = baseline_scores.mean()

print(f"Baseline AUC (5‐fold CV) with all core features: {baseline_mean:.4f}")
print("Individual fold AUCs:", np.round(baseline_scores, 4))

Baseline AUC (5‐fold CV) with all core features: 0.9392
Individual fold AUCs: [0.9385 0.9379 0.9367 0.9415 0.9413]


In [7]:
ablation_results = []

for f in core_features:
    # Create a reduced feature matrix without column f
    X_sub = X.drop(columns=[f])
    
    # Recompute CV‐AUC on this reduced set
    scores = cross_val_score(clf, X_sub, y, cv=5, scoring="roc_auc")
    mean_score = scores.mean()
    
    # ΔAUC: how much performance drops (positive => feature was helpful)
    delta = baseline_mean - mean_score
    
    ablation_results.append({
        "feature": f,
        "cv_auc_without_feature": mean_score,
        "delta_auc": delta
    })

# Convert to DataFrame and sort by delta_auc (ascending)
ablation_df = pd.DataFrame(ablation_results)
ablation_df = ablation_df.sort_values(by="delta_auc", ascending=True).reset_index(drop=True)

# Display the result
ablation_df


,feature,cv_auc_without_feature,delta_auc
0,burstiness_token_var,0.939841,-0.000641
1,burstiness_token_std,0.939831,-0.000631
2,repeated_2gram_ratio,0.939226,-0.000025
3,repeated_3gram_ratio,0.939028,0.000173
4,unique_2grams,0.938421,0.000779
5,unique_3grams,0.938306,0.000894
6,avg_word_length,0.938251,0.000950
7,sentence_length_entropy,0.937718,0.001482
8,flesch_reading_ease,0.937069,0.002131
9,punctuation_ratio,0.936452,0.002748


In [8]:
# Flag features that can be dropped: delta_auc <= 0
to_drop = ablation_df.loc[ablation_df["delta_auc"] <= 0, "feature"].tolist()
print("Features you can consider removing (ΔAUC ≤ 0):")
print(to_drop)


Features you can consider removing (ΔAUC ≤ 0):
['burstiness_token_var', 'burstiness_token_std', 'repeated_2gram_ratio']


In [9]:
pruned_features = [f for f in core_features if f not in to_drop]
X_pruned = X[pruned_features]

# 5‐fold CV on the pruned feature set
scores_pruned = cross_val_score(clf, X_pruned, y, cv=5, scoring="roc_auc")
pruned_mean = scores_pruned.mean()

print(f"Pruned AUC (5‐fold CV) with {len(pruned_features)} features: {pruned_mean:.4f}")
print("Individual fold AUCs:", np.round(scores_pruned, 4))
print(f"AUC change: {baseline_mean - pruned_mean:.4f}")

Pruned AUC (5‐fold CV) with 12 features: 0.9391
Individual fold AUCs: [0.9395 0.9381 0.9353 0.9403 0.942 ]
AUC change: 0.0001


In [10]:
ablation_df.to_csv("ablation_results.csv", index=False)
print("Saved ablation results to 'ablation_results.csv'")

Saved ablation results to 'ablation_results.csv'


In [11]:
with open("pruned_features.txt", "w") as f:
    for feat in pruned_features:
        f.write(feat + "\n")
print("Saved pruned feature list to 'pruned_features.txt'")

Saved pruned feature list to 'pruned_features.txt'
